In [7]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchResults

llm=ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)

class CompanyOverviewToolArgsSchema(BaseModel):
    symbol : str = Field(
        description="Stock symbol of the company. Example: AAPL, TSLA",
    )

class CompanyOverviewTool(BaseTool):
    name="CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name="CompanyIncomeStatement"
    description="""
    Use this to get the income statement of company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]

class CompanyStockPerformanceTool(BaseTool):
    name="CompanyStockPerformance"
    description="""
    Use this to get the weekly performance fo a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools = [
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)




> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`


[snippet: First, a refresher — Cloudflare operates Unimog, a Layer 4 load balancer that handles all incoming Cloudflare traffic. Cloudflare's network uses anycast, which allows a single IP address to route requests to a variety of different locations. For most Cloudflare services with anycast, the given IP address will route to the nearest Cloudflare ..., title: Our container platform is in production. It has GPUs. Here's an early look, link: https://blog.cloudflare.com/container-platform-preview/], [snippet: Cloudflare, Inc. (NYSE: NET) is the leading connectivity cloud company on a mission to help build a better Internet. It empowers organizations to make their employees, applications and networks faster and more secure everywhere, while reducing complexity and cost. Cloudflare's connectivity cloud delivers the most full-featured, unified ..., title: Cloudflare Announces 

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': '### Cloudflare Inc. (NYSE: NET) Financial Overview\n\n#### Company Overview\n- **Name**: Cloudflare Inc.\n- **Market Capitalization**: $29.92 billion\n- **Sector**: Technology\n- **Industry**: Services - Prepackaged Software\n- **Headquarters**: 101 Townsend St., San Francisco, CA, USA\n- **Official Website**: [Cloudflare](https://www.cloudflare.com)\n\n#### Key Financial Metrics (Latest Quarter Ending June 30, 2024)\n- **Revenue (TTM)**: $1.48 billion\n- **Gross Profit (TTM)**: $989.74 million\n- **Operating Income**: -$185.49 million\n- **Net Income**: -$183.95 million\n- **EBITDA**: -$48.08 million\n- **Earnings Per Share (EPS)**: -$0.30\n- **Profit Margin**: -6.9%\n- **Operating Margin**: -8.65%\n- **Return on Assets (ROA)**: -3.59%\n- **Return on Equity (ROE)**: -13.4%\n- **Quarterly R